what is COD: Hernandez & Jones 2024
- type 1: COD could occur both when the player starts from a static or semi-static position and must move into a different direction (does not involve deceleration)
- type 2: advances in a certain direction prior to having to maneuver into a new direction (does often involve deceleration)
- Type 3: a change in the initial path without changing the direction that the player is facing with a combination of linear (backwards or forward) and lateral movements (does always involve deceleration)
- type 4: arc run or curvilinear type run
curvilinear type run


possible modules:
math.athan2() --> returns the arc tangent of y/x, in radians


classifications idee?:
- characterised the COD angle as one of ‘0–45 DEG’, ‘45–90 DEG’, ‘90–135 DEG’ and ‘135–180 DEG’

COD duration:
- Brenda et al 2022: 1s
- Reilly et al. 2021: averagedurationbetweenthe‘StartTime’and‘EndTime’ forthe observedCODwas3.1s

In [ ]:
import numpy as np
import pandas as pd

def compute_angle_changes(df, player_name):
    x = df[f"{player_name}_x"].values
    y = df[f"{player_name}_y"].values
    
    # Compute velocity vectors
    dx = np.diff(x)
    dy = np.diff(y)
    v = np.stack((dx, dy), axis=1) # combines dx and dy into a 2D array where each row is a 2D velocity vector

    # Normalize to avoid dividing by zero
    norms = np.linalg.norm(v, axis=1) # Computes the Euclidean norm (magnitude) of each 2D velocity vector
    valid = norms[:-1] > 0  # boolean mask identifying where the first velocity vector (v[:-1]) is non-zero.
    v1 = v[:-1][valid] # v1: The velocity at time t
    v2 = v[1:][valid] # v1: The velocity at time t + 1

    # Compute angle changes (in radians)
    dot_products = np.einsum('ij,ij->i', v1, v2) # ompact and efficient way to compute row-wise dot products.
    norms_product = np.linalg.norm(v1, axis=1) * np.linalg.norm(v2, axis=1) # Calculates the product of magnitudes (‖v1‖‖v2‖) for each vector pair.
    cos_theta = np.clip(dot_products / norms_product, -1.0, 1.0) # Computes the cosine of the angle between v1 and v2
    angles = np.arccos(cos_theta) # Computes the angle (in radians) between the velocity vectors.

    return angles  # or convert to degrees with np.degrees(angles)

# Example usage:
angles_amuzu = compute_angle_changes(df, "Amuzu")

In [ ]:
import numpy as np
import pandas as pd

# Assuming df has 'X' and 'Y' columns with consecutive player positions
positions = df[["X", "Y"]].to_numpy()

# Compute displacement vectors
vectors = np.diff(positions, axis=0)

# Normalize vectors
norms = np.linalg.norm(vectors, axis=1, keepdims=True)
unit_vectors = vectors / norms

# Compute angle between successive unit vectors
cos_angles = np.sum(unit_vectors[:-1] * unit_vectors[1:], axis=1)
angles = np.arccos(np.clip(cos_angles, -1.0, 1.0))  # In radians
angles_deg = np.degrees(angles)

# Detect change of direction where angle > threshold (e.g., 30°)
change_points = np.where(angles_deg > 30)[0] + 1  # +1 to get the index of the second vector in the pair

print("Change of direction at indices:", change_points)

The result is an array of angle changes per frame (except first two).

If your data has missing values (NaNs), consider interpolating or masking before processing.

You can loop over all player names like so:

In [ ]:
players = [col[:-2] for col in df.columns if col.endswith('_x')]
angle_changes_by_player = {
    player: compute_angle_changes(df, player) for player in players
}